In [13]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

In [14]:
n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[1].Diff(z)- u[2].Diff(y), u[2].Diff(x)- u[0].Diff(z), u[0].Diff(y)- u[1].Diff(x)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF(      (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 



In [15]:
def RunP(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurl = HCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        start = time.time()
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        PEAK = CF ( (peak, 0, 0 ), dims=(3,1))
        CurlCurlPeak = CF( Curl(Curl(PEAK)), dims=(3,3) )

        # grid functions 
        gfEnm1 = GridFunction(fesHCurl) 
        gfEnm1.Set ( PEAK, bonus_intorder=15, dual=True)

        gfCurl = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=15, dual=True)

        end = time.time()
        print("Time coeff and grid functions :"+str(end-start))
        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################
        start = time.time()
        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        a += InnerProduct(u,v)*dx 


        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        

        end = time.time()
        print("Time define linear and blinear form :"+str(end-start))

        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################

        start = time.time()
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()


        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1500,tol=1e-9)
        gfCurlCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        


        end = time.time()
        print("Assemble and find inverse :"+str(end-start))

        start = time.time()
        print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 

        end = time.time()
        print("time to draw :"+str(end-start))
        return Error

In [16]:
Maxh = 0.10
Order = 3
with TaskManager():
    RunP(Maxh, Order)


Time coeff and grid functions :10.219223737716675
Time define linear and blinear form :0.002992391586303711
CG NOT converged in 1500 iterations to residual 1.1730242252478782e-07
Assemble and find inverse :117.6665518283844
error of my inc
Error inc(PEAK)-gfcurlCurlG when order is 3-->0.013797239054358222


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

time to draw :8.55838394165039
